# Conversation Summary
Now let's take a look at using a slightly more complex type of memory - ConversationSummaryMemory.  
This type of memory creates a summary of the conversation over time.  
This can be useful for condensing information from the conversation over time.  
Conversation summary memory summarizes the conversation as it happens  
and stores the current summary in memory. 

This memory can then be used to inject the summary of the conversation so far into a prompt/chain.  
This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

Let's first explore the basic functionality of this type of memory.

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

In [ ]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [ ]:
memory.load_memory_variables({})

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [ ]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)

In [ ]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "I am fine, you?"}, {"output": "good"})

In [ ]:
memory.load_memory_variables({})

We can also utilize the predict_new_summary method directly.

In [ ]:
messages = memory.chat_memory.messages
print(messages)

In [ ]:
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

## Initializing with messages/existing summary
If you have messages outside this class,  
you can easily initialize the class with ChatMessageHistory.  

During loading, a summary will be calculated.

In [ ]:
history = ChatMessageHistory()

In [ ]:
# When used for updating history, users should favor usage of `add_messages` over `add_message` or other variants like `add_user_message` and `add_ai_message` to avoid unnecessary round-trips to the underlying persistence layer.

history.add_user_message("hi")
history.add_ai_message("hi there!")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
history.add_message(HumanMessage("hi"))
history.add_message(AIMessage("hi there!"))
# history.add_message(SystemMessage("summarize the conversation"))

In [ ]:
memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)

In [ ]:
history

In [ ]:
memory.buffer

Optionally you can speed up initialization using a previously generated summary, and avoid regenerating the summary by just initializing directly.

In [ ]:
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)

In [ ]:
# memory.memory_variables

In [ ]:
memory.buffer

## Using in a chain
Let's walk through an example of using this in a chain, again setting verbose=True so we can see the prompt.

In [ ]:
from langchain_openai import OpenAI
from langchain.chains.conversation.base import ConversationChain

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

In [ ]:
conversation_with_summary.predict(input="Tell me more about it!")

In [ ]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")